In [1]:
from deepface import DeepFace

2023-04-12 11:06:51.295415: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-12 11:06:52.225090: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [25]:
import numpy as np

In [5]:
import os,glob

In [17]:
files = glob.glob('face/*.jpg')

In [18]:
files

['face/mohsin.jpg',
 'face/nazar.jpg',
 'face/anwaa.jpg',
 'face/kumail.jpg',
 'face/ali.jpg',
 'face/farooq.jpg',
 'face/raaziq.jpg']

In [21]:
representations = []
for img_path in files:     
    embedding = DeepFace.represent(img_path = img_path, model_name = "Facenet")[0]["embedding"]
    representation = []
    representation.append(img_path)
    representation.append(embedding)
    representations.append(representation)

In [26]:
embeddings = []
for i in range(0, len(representations)):
    embedding = representations[i][1]
    embeddings.append(embedding)
 
embeddings = np.array(embeddings, dtype='f')

In [29]:
	
target_representation = DeepFace.represent(img_path = "ali_.jpg", model_name = "Facenet")[0]["embedding"]

In [31]:
target_representation = np.array(target_representation, dtype='f')
target_representation = np.expand_dims(target_representation, axis=0)

In [32]:
import faiss

In [33]:
dimensions = 128 #FaceNet output is 128 dimensional vector
 
metric = 'euclidean' #euclidean, cosine
 
if metric == 'euclidean':
    index = faiss.IndexFlatL2(dimensions)
elif metric == 'cosine':
    index = faiss.IndexFlatIP(dimensions)
    faiss.normalize_L2(embeddings)

In [34]:
	
index.add(embeddings)

In [35]:
#save
faiss.write_index(index,"vector.index")
 
#restore
index = faiss.read_index("vector.index")

In [36]:
k = 3
distances, neighbors = index.search(target_representation, k)

In [38]:
distances,neighbors

(array([[ 55.309193, 235.87521 , 238.27356 ]], dtype=float32),
 array([[4, 1, 6]]))

In [43]:
	
target_representation = DeepFace.represent(img_path = "face/saad.jpg", model_name = "Facenet")[0]["embedding"]
target_representation = np.array(target_representation, dtype='f')
target_representation = np.expand_dims(target_representation, axis=0)


In [44]:
index.add(target_representation)

In [45]:
k = 3
target_representation = DeepFace.represent(img_path = "saad_.jpg", model_name = "Facenet")[0]["embedding"]
target_representation = np.array(target_representation, dtype='f')
target_representation = np.expand_dims(target_representation, axis=0) 
distances, neighbors = index.search(target_representation, k)

In [46]:
distances,neighbors

(array([[ 43.374462, 193.63547 , 198.33301 ]], dtype=float32),
 array([[7, 6, 2]]))

In [50]:
total_distances = (1/distances).sum()

In [59]:
distance_metrics = [(1/d)/total_distances for d in distances]

In [60]:
distance_metrics

[array([0.6931469 , 0.15526532, 0.15158784], dtype=float32)]